<a href="https://colab.research.google.com/github/dagi290/graph/blob/main/AI_project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -q pandas numpy scikit-learn
!pip install -q surprise gradio streamlit
!pip install -q flask scipy ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.7 MB/s eta 0:00:00


In [3]:

import pandas as pd
import numpy as np
import zipfile
import requests
import os
from google.colab import drive



In [2]:
import pandas as pd
movies = pd.read_csv('/content/netflixData.csv')

In [4]:
names=['movie_id', 'title', 'release_date', 'video_release_date',
                            'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation',
                            'Children', 'Comedy', 'Crime', 'Documentary', 'Drama',
                            'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
                            'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [6]:
# Create movie title dictionary
movie_titles = movies.set_index('Show Id')['Title'].to_dict()

# The 'ratings' DataFrame is not defined, so this line will be commented out or removed
# print(f"Ratings: {ratings.shape}")
print(f"Movies: {movies.shape}")
print("\nSample movies:")
print(movies[['Show Id', 'Title']].head())

Movies: (5967, 13)

Sample movies:
                                Show Id                          Title
0  cc1b6ed9-cf9e-4057-8303-34577fb54477                       (Un)Well
1  e2ef4e91-fb25-42ab-b485-be8e3b23dedb                         #Alive
2  b01b73b7-81f6-47a7-86d8-acb63080d525  #AnneFrank - Parallel Stories
3  b6611af0-f53c-4a08-9ffa-9716dc57eb9c                       #blackAF
4  7f2d4170-bab8-4d75-adc2-197f7124c070               #cats_the_mewvie


In [13]:
from google.colab import drive
import os
# Mount Google Drive
drive.mount('/content/drive')

# Define the directory path
drive_path = '/content/drive/MyDrive/Colab_Projects/movie_recommender'

# Create the directory if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# Save to Drive for persistence
# ratings.to_csv(f'{drive_path}/ratings.csv', index=False)
movies.to_csv(f'{drive_path}/movies.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Cell 3: Build multiple recommendation models
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler
import pickle
import pandas as pd
import numpy as np

# --- FIX START: Define 'ratings' DataFrame and adjust MovieRecommender for 'Show Id'/'Title' ---
# The error 'NameError: name 'ratings' is not defined' means the ratings DataFrame is missing.
# Since 'movies' is from netflixData.csv (which typically does not contain user ratings),
# a separate ratings dataset is needed. For demonstration, we'll create a dummy ratings DataFrame.

# Get existing movie 'Show Id's from the 'movies' DataFrame
# Ensure movies DataFrame is accessible (it's a kernel variable).
if 'Show Id' not in movies.columns:
    raise ValueError("The 'movies' DataFrame must contain a 'Show Id' column.")

available_movie_ids = movies['Show Id'].unique()

# Create dummy ratings data: Each user rates a random subset of movies
# This is a placeholder and should be replaced with actual ratings data.
num_users = 100
user_ids = np.arange(1, num_users + 1)
dummy_ratings_data = []

for user_id in user_ids:
    # Each user rates between 5 and 20 movies
    num_ratings = np.random.randint(5, 21)
    # Select random movie IDs that are actually in the movies DataFrame
    rated_movie_ids = np.random.choice(available_movie_ids, min(num_ratings, len(available_movie_ids)), replace=False)
    for movie_id in rated_movie_ids:
        # Assign random ratings between 1 and 5
        rating = np.random.randint(1, 6)
        dummy_ratings_data.append({'user_id': user_id, 'movie_id': movie_id, 'rating': rating})

ratings = pd.DataFrame(dummy_ratings_data)
print(f"Dummy ratings DataFrame created with shape: {ratings.shape}")
print("First 5 rows of dummy ratings:")
print(ratings.head())

# --- MovieRecommender class adjusted for 'Show Id' and 'Title' ---
class MovieRecommender:
    def __init__(self, ratings_df, movies_df):
        self.ratings = ratings_df
        self.movies = movies_df
        self.similarity_matrix = None
        self.popular_movies = None

    def prepare_data(self):
        """Prepare data for collaborative filtering"""
        # Create user-item matrix
        self.user_movie_matrix = self.ratings.pivot_table(
            index='user_id',
            columns='movie_id',
            values='rating'
        ).fillna(0)

        # Calculate movie popularity
        self.popular_movies = self.ratings.groupby('movie_id')['rating'].agg(['count', 'mean']).reset_index()
        self.popular_movies.columns = ['movie_id', 'rating_count', 'rating_mean']

        # Merge with movie titles - adjusted to use 'Show Id' and 'Title' from the movies DataFrame
        self.popular_movies = self.popular_movies.merge(
            self.movies[['Show Id', 'Title']], # Use 'Show Id' and 'Title'
            left_on='movie_id', right_on='Show Id' # Merge on appropriate keys
        )
        self.popular_movies.drop(columns=['Show Id'], inplace=True) # Drop redundant 'Show Id'

        # Sort by popularity (rating count * mean rating)
        self.popular_movies['popularity_score'] = (
            self.popular_movies['rating_count'] * self.popular_movies['rating_mean']
        )
        self.popular_movies = self.popular_movies.sort_values('popularity_score', ascending=False)

        # Prepare content-based features - adjusted for typical netflixData.csv genre column
        # Assuming 'Listed In' or 'Genre' is the genre column in netflixData.csv.
        # Fallback to 'Description' if neither is found.
        if 'Listed In' in self.movies.columns:
            self.movies['genres'] = self.movies['Listed In'].fillna('')
        elif 'Genre' in self.movies.columns:
            self.movies['genres'] = self.movies['Genre'].fillna('')
        elif 'Description' in self.movies.columns:
            self.movies['genres'] = self.movies['Description'].fillna('')
        else:
            # If no obvious genre column, create an empty one to avoid errors.
            print("Warning: No explicit genre column ('Listed In', 'Genre', 'Description') found. Content-based recommendations might be limited.")
            self.movies['genres'] = ''

    def build_content_based_model(self):
        """Build content-based filtering model"""
        # Create TF-IDF matrix for genres
        tfidf = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf.fit_transform(self.movies['genres'])

        # Compute cosine similarity
        self.content_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

        # Create movie index mapping - adjusted to use 'Title'
        self.movie_idx = {title: idx for idx, title in enumerate(self.movies['Title'])}
        self.idx_movie = {idx: title for title, idx in self.movie_idx.items()}

        return self

    def build_collaborative_filtering(self):
        """Build collaborative filtering model"""
        # Dimensionality reduction for sparse matrix
        svd = TruncatedSVD(n_components=50, random_state=42)
        reduced_matrix = svd.fit_transform(self.user_movie_matrix.T)

        # Compute similarity
        self.collab_similarity = cosine_similarity(reduced_matrix)

        return self

    def get_similar_movies_content(self, movie_title, n=10):
        """Get similar movies based on content"""
        if movie_title not in self.movie_idx:
            print(f"Movie '{movie_title}' not found in content-based index.")
            return []

        idx = self.movie_idx[movie_title]
        sim_scores = list(enumerate(self.content_similarity[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]

        return [self.idx_movie[i[0]] for i in sim_scores]

    def get_similar_movies_collab(self, movie_id_for_lookup, n=10):
        """Get similar movies based on collaborative filtering"""
        # movie_id_for_lookup here refers to 'Show Id' from our dummy ratings and movies['Show Id']
        if movie_id_for_lookup not in self.user_movie_matrix.columns:
            print(f"Movie ID '{movie_id_for_lookup}' not found in collaborative filtering matrix.")
            return []

        movie_idx = list(self.user_movie_matrix.columns).index(movie_id_for_lookup)
        sim_scores = list(enumerate(self.collab_similarity[movie_idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:n+1]

        movie_indices = [i[0] for i in sim_scores]
        movie_ids_from_matrix = [self.user_movie_matrix.columns[idx] for idx in movie_indices]

        return [movie_titles[mid] for mid in movie_ids_from_matrix if mid in movie_titles]

    def get_popular_movies(self, n=10):
        """Get popular movies"""
        return self.popular_movies['Title'].head(n).tolist()

    def hybrid_recommendation(self, movie_title, n=10):
        """Hybrid recommendation combining content and collaborative"""
        content_rec = self.get_similar_movies_content(movie_title, n*2)

        # Try to get movie 'Show Id' for collaborative filtering
        movie_id_for_collab = self.movies[self.movies['Title'] == movie_title]['Show Id'].values
        collab_rec = []
        if len(movie_id_for_collab) > 0:
            collab_rec = self.get_similar_movies_collab(movie_id_for_collab[0], n*2)

        # Combine and deduplicate
        all_rec = list(dict.fromkeys(content_rec + collab_rec))
        return all_rec[:n]

    def save_model(self, path):
        """Save model to file"""
        with open(path, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load_model(cls, path):
        """Load model from file"""
        with open(path, 'rb') as f:
            return pickle.load(f)

# Create and train the recommender
print("Building recommendation models...")
recommender = MovieRecommender(ratings, movies)
recommender.prepare_data()
recommender.build_content_based_model()
recommender.build_collaborative_filtering()

# Test the model
# Use an existing movie title from the loaded data for testing
# Ensure that the movie has been rated in the dummy dataset for collab recommendations to work
# Or, select a popular movie if a random one causes issues.
if not movies.empty:
    test_movie = movies['Title'].sample(1).iloc[0]
else:
    test_movie = "(Un)Well" # Fallback if movies is empty

print(f"\nTesting with: {test_movie}")
print("Content-based recommendations:")
print(recommender.get_similar_movies_content(test_movie, 5))
print("\nPopular movies:")
print(recommender.get_popular_movies(5))

# Save model to Drive
recommender.save_model('/content/drive/MyDrive/Colab_Projects/movie_recommender/model.pkl')
print("\n✅ Model saved to Google Drive!")

Dummy ratings DataFrame created with shape: (1322, 3)
First 5 rows of dummy ratings:
   user_id                              movie_id  rating
0        1  7b2cb3e5-2379-4a2e-a866-789c680d1238       1
1        1  99b34f70-2d39-408c-8d57-f98b4204eb29       5
2        1  54de74ec-42e6-48f0-b5de-ba2f6e21cf61       2
3        1  24d4d305-425f-48d5-b2a4-7e656756aba2       4
4        1  aa0b520c-573f-41a1-a4c3-fb5df019feee       4
Building recommendation models...

Testing with: Danur: I Can See Ghosts
Content-based recommendations:
['The Writer', 'Marianne', "Let's Fight Ghost", 'The Land', 'Oh My Ghost 4']

Popular movies:
["Monty Python's Life of Brian", 'How I Became a Superhero', 'Eastsiders', 'Ordinary Heroes', 'The Chase']

✅ Model saved to Google Drive!
